In [ ]:
import sys
import ipywidgets as widgets
from IPython.display import display
from openmm.app import PDBFile, ForceField, Simulation, Modeller, GromacsGroFile, GromacsTopFile
from openmm import LangevinIntegrator, VerletIntegrator, Platform
from openmm.unit import kelvin, picosecond, femtosecond, nanometers
from openmm.app import PDBReporter, StateDataReporter, DCDReporter, PME, HBonds
import numpy as np

class VelocityReporter(object):
    def __init__(self, file, reportInterval):
        self._out = open(file, 'w')
        self._reportInterval = reportInterval

    def __del__(self):
        self._out.close()

    def describeNextReport(self, simulation):
        steps = self._reportInterval - simulation.currentStep%self._reportInterval
        return {'steps': steps, 'periodic': None, 'include':['velocities']}

    def report(self, simulation, state):
        velocities = state.getVelocities().value_in_unit(nanometers/picosecond)
        for v in velocities:
            self._out.write('%g %g %g\n' % (v[0], v[1], v[2]))

class InteractiveSimulation:
        def __init__(self):
                # Default OpenMM force fields
                self.openmm_force_fields = ['amber14-all.xml', 'amber14/tip3pfb.xml']

                # Step 1: Create input widgets
                self.input_file_w = widgets.Text(
                        value='/storage_common/angiod/MB-Fit-Data-Forge/POPC/md.simulation/prod.gro',
                        placeholder='Enter the input file path (PDB or GRO)',
                        description='Input File:',
                        disabled=False
                )
                self.input_top_w = widgets.Text(
                        value='/storage_common/angiod/MB-Fit-Data-Forge/POPC/md.simulation/prod.top',
                        placeholder='Enter the topology file path (optional, only for GRO files)',
                        description='Topology File:',
                        disabled=False
                )
                self.add_water_w = widgets.Checkbox(
                        value=False,
                        description='Add Water Molecules',
                        disabled=False
                )
                self.temperature_w = widgets.FloatText(
                        value=298.15,
                        description='Temperature (K):',
                        disabled=False
                )
                self.time_step_w = widgets.FloatText(
                        value=1.0,
                        description='Time Step (fs):',
                        disabled=False
                )
                self.num_steps_w = widgets.IntText(
                        value=1000,
                        description='Simulation Steps:',
                        disabled=False
                )
                self.reporter_step_w = widgets.IntText(
                        value=1,
                        description='Reporter Step:',
                        disabled=False
                )
                self.output_file_w = widgets.Text(
                        value='output',
                        placeholder='Enter the output file name (without extension)',
                        description='Output File:',
                        disabled=False
                )
                self.output_format_w = widgets.Dropdown(
                        options=['multipdb', 'pdb+dcd+trr'],
                        value='pdb+dcd+trr',
                        description='Output Format:',
                        disabled=False
                )
                self.run_button = widgets.Button(
                        description='Confirm setup',
                        button_style='success'
                )
                self.output_area = widgets.Output()

                # Step 3: Attach the callback to the button
                self.run_button.on_click(self.run_simulation_callback)

                # Step 4: Display the widgets
                display(widgets.VBox([
                        self.input_file_w,
                        self.input_top_w,
                        self.add_water_w,
                        self.temperature_w,
                        self.time_step_w,
                        self.num_steps_w,
                        self.reporter_step_w,
                        self.output_file_w,
                        self.output_format_w,
                        self.run_button,
                        self.output_area
                ]))

        # Step 2: Define the callback function
        def run_simulation_callback(self, b):
                with self.output_area:
                        self.output_area.clear_output()
                        try:
                                # Collect input parameters
                                self.params = {
                                        'input_file': self.input_file_w.value.strip(),
                                        'input_top': self.input_top_w.value.strip(),
                                        'add_water': self.add_water_w.value,
                                        'temperature': self.temperature_w.value * kelvin,
                                        'time_step': self.time_step_w.value * femtosecond,
                                        'num_steps': self.num_steps_w.value,
                                        'reporter_step': self.reporter_step_w.value,
                                        'output_file_base': self.output_file_w.value.strip(),
                                        'output_format': self.output_format_w.value
                                }
                                return self.params
                        except Exception as e:
                                print(f"Error: {e}")

        def run(self):
                input_file = self.params.get('input_file')
                input_top = self.params.get('input_top')
                add_water = self.params.get('add_water')
                temperature = self.params.get('temperature')
                time_step = self.params.get('time_step')
                num_steps = self.params.get('num_steps')
                reporter_step = self.params.get('reporter_step')
                output_file_base = self.params.get('output_file_base')
                output_format = self.params.get('output_format')

                if input_file.endswith('.gro') and input_top:
                        # GROMACS input
                        gro = GromacsGroFile(input_file)
                        top = GromacsTopFile(input_top, periodicBoxVectors=gro.getPeriodicBoxVectors(), includeDir='/apps/gromacs_2020.6/share/gromacs/top')
                        system = top.createSystem(nonbondedMethod=PME, nonbondedCutoff=1*nanometers, constraints=HBonds)
                        topology  = top.topology
                        positions = gro.positions
                else:
                        # PDB input
                        pdb = PDBFile(input_file)
                        forcefield = ForceField(*self.openmm_force_fields)
                        modeller = Modeller(pdb.topology, pdb.positions)
                        if add_water:
                                modeller.addSolvent(forcefield)
                        system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME, nonbondedCutoff=1*nanometers, constraints=HBonds)
                        topology = modeller.topology
                        positions = modeller.positions

                # Create an integrator
                integrator = VerletIntegrator(time_step)
                # friction_coeff = 0.1 / picosecond
                # integrator = LangevinIntegrator(temperature, friction_coeff, time_step)

                # Create a simulation
                platform = Platform.getPlatformByName('CUDA')
                simulation = Simulation(topology, system, integrator, platform=platform)
                simulation.context.setPositions(positions)

                # Reporters
                if output_format == 'multipdb':
                        output_file = f"{output_file_base}.pdb"
                        simulation.reporters.append(PDBReporter(output_file, reporter_step))
                elif output_format == 'pdb+dcd+trr':
                        dcd_output_file = f"{output_file_base}.dcd"
                        vel_output_file = f"{output_file_base}.vel"
                        simulation.reporters.append(DCDReporter(dcd_output_file, reporter_step))
                        simulation.reporters.append(VelocityReporter(vel_output_file, reporter_step))
                else:
                        raise ValueError("Unsupported output format.")

                simulation.reporters.append(StateDataReporter(sys.stdout, reporter_step, step=True, potentialEnergy=True, temperature=True))

                # Set initial velocities and run simulation
                simulation.context.setVelocitiesToTemperature(temperature)
                simulation.step(num_steps)
                print(f"Simulation completed. Output saved to {output_file_base} with format {output_format}")
        
        def convert(self):
                output_format = self.params.get('output_format')
                # Convert DCD to TRR if pdb+dcd+trr is selected
                if output_format == 'pdb+dcd+trr':
                        import MDAnalysis as mda
                        from MDAnalysis.coordinates.TRR import TRRWriter

                        input_file = self.params.get('input_file')
                        output_file_base = self.params.get('output_file_base')
                        dcd_output_file = f"{output_file_base}.dcd"
                        vel_output_file = f"{output_file_base}.vel"
                        pdb_output_file = f"{output_file_base}.pdb"
                        trr_output_file = f"{output_file_base}.trr"

                        u = mda.Universe(input_file, dcd_output_file)  # Load the topology and trajectory
                        # Save the first frame as a PDB file
                        with mda.Writer(pdb_output_file, n_atoms=u.atoms.n_atoms) as pdb_writer:
                                pdb_writer.write(u.atoms)
                        print(f"First frame saved as PDB. Output saved to {pdb_output_file}")

                        # Read velocities from the velocity file
                        velocities = []
                        with open(vel_output_file, 'r') as vel_file:
                                for line in vel_file:
                                        velocities.append([float(v) for v in line.split()])
                        velocities = np.array(velocities).reshape(len(u.trajectory), u.atoms.n_atoms, 3)  # Reshape to (n_frames, n_atoms, 3)

                        # Write TRR file with velocities
                        with TRRWriter(trr_output_file, n_atoms=u.atoms.n_atoms) as trr_writer:
                                for ts, vel in zip(u.trajectory, velocities):
                                        ts.has_velocities = True
                                        u.atoms.velocities = vel  # Set velocities for the current timestep
                                        trr_writer.write(u)
                        print(f"Converted DCD to TRR with velocities. Output saved to {trr_output_file}")


interactive_simulation = InteractiveSimulation()

In [2]:
interactive_simulation.run()

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
1,7969.264486856846,264.40067542656135
2,8863.094687052158,255.1681770867739
3,10075.051268167881,242.66756361842653
4,11376.130293436436,229.2728573227704
5,12488.969374246983,217.82941309233397
6,13159.010206766514,210.94060610349874
7,13231.922163553623,210.19054516285667
8,12700.2546449379,215.6458782708749
9,11711.019850321201,225.80122807266284
10,10526.247107096104,237.96739226073743
11,9453.98341233292,248.98106552576724
12,8767.194410868076,256.0388263518034
13,8632.488913126377,257.43490133351014
14,9064.849424906162,253.01160315939725
15,9924.628323145298,244.20336850283078
16,10954.18882615128,233.65661965572883
17,11846.742491312412,224.51712124267308
18,12330.902477808384,219.5680703812574
19,12246.687616892246,220.44843660664517
20,11592.848055429844,227.16555916873216
21,10532.70258194962,238.0440910259042
22,9351.735064096836,250.1570431660252
23,8376.666662760166,260.15975678063734
24,7877.069002695469,265.2862419

In [3]:
interactive_simulation.convert()

/home/angiod@usi.ch/anaconda3/envs/geqmd/lib/python3.10/site-packages/MDAnalysis/coordinates/DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"
/home/angiod@usi.ch/anaconda3/envs/geqmd/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'altLocs' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/home/angiod@usi.ch/anaconda3/envs/geqmd/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'icodes' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/home/angiod@usi.ch/anacond

First frame saved as PDB. Output saved to /storage_common/angiod/MB-Fit-Data-Forge/Tripalmitin/md_simulations/POPC.verlet.298K.pdb
Converted DCD to TRR with velocities. Output saved to /storage_common/angiod/MB-Fit-Data-Forge/Tripalmitin/md_simulations/POPC.verlet.298K.trr
